In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.relativedelta
from re import sub
import os.path
# import pypyodbc as odbc
import warnings

In [2]:
pd.options.mode.copy_on_write = True
warnings.simplefilter(action='ignore', category=FutureWarning)

# Test database

In [31]:
# drive_name = 'SQL SERVER'
# server_name = 'DESKTOP-29355SM\SQLEXPRESS'
# database_name = 'Bank'

# connection_string = f"""
#     DRIVER={{{drive_name}}};
#     SERVER={server_name};
#     DATABASE={database_name};
#     Trust_Connection=yes;
# """

# conn = odbc.connect(connection_string)

In [32]:
# cursor = conn.cursor()

In [33]:
# cursor.execute("""
# Create table Persons (
#     PersonID int,
#     LastName varchar(255),
#     FirstName varchar(255),
#     Address varchar(255),
#     City varchar(255)
# )
# """)

In [34]:
# df = pd.read_sql('Select * from account', conn)

In [35]:
# conn.commit()

# Function

In [3]:
def extract_date_review(text_date, crawl_date): # Biến đổi khoảng thời gian (1 tháng trước, 2 ngày trước) thằnh ngày cụ thể và trả về datetime
    sub_text = text_date.split(' ')
    date_cmt = ''

    if 'năm' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(t))
    elif 'tháng' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(t))
    elif 'tuần' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(weeks=int(t))
    elif 'ngày' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(days=int(t))
    else:
        date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date()

    return str(date_cmt)

In [4]:
def create_id(last_id): # Tạo ID mới và trả về string
    last_id_list = []
    newest_id = ''

    for s in list(last_id):
        if s.isnumeric():
            last_id_list.append(s)
        else:
            newest_id += s
    
    last_id = str(int(''.join(last_id_list)) + 1)
    if len(last_id_list) != len(last_id):
        num_0 = len(last_id_list) - len(last_id)
        newest_id += '0'*num_0 + last_id
    else:
        newest_id += last_id
    
    return newest_id

In [6]:
def add_data(file_table_name, df, id_format=None): # Thêm df vào file save
    # Lưu ý: df không chứa cột ID, thứ tự cột và định dạng giống với file save và nếu file save trống thì cần đưa ID format (S000, PL00,...)
    table = pd.read_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name))

    # Loại bỏ những dòng trong df đã có trong table
    drop_index = []
    for index in range(len(df)):
        if list(df.iloc[index]) in table[table.columns[1:]].values.tolist():
            drop_index.append(index)
    df.drop(drop_index, inplace=True)

    newest_id = ''
    if id_format != None:
        newest_id = create_id(id_format)
    else:
        newest_id = create_id(table.iloc[-1][0])

    list_id = []
    for i in range(len(df)):
        list_id.append(newest_id)
        newest_id = create_id(newest_id)
    df.insert(loc=0, column='ID', value=list_id)

    df.columns = table.columns
    table = pd.concat([table, df], ignore_index=True)
    table.to_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name), index=False)

In [7]:
def add_shop(new_shop_df): # Thêm shop mới vào file Shop.csv (new_shop_df chỉ bao gồm 'Shop name', 'Shop rating' và đặt đúng thứ tự)
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    check_shop_list = new_shop_df.drop_duplicates(subset=new_shop_df.columns[0], keep='last', ignore_index=True)
    drop_index = []

    for index_shop in range(len(check_shop_list)):
        if check_shop_list.iloc[index_shop][0] in shop_df['ShopName'].values:
            drop_index.append(index_shop)

    check_shop_list.drop(drop_index, inplace=True)

    if len(shop_df) == 0:
        add_data('Shop.csv', check_shop_list, id_format='S000')
    else:
        add_data('Shop.csv', check_shop_list)

# Main

In [103]:
platform_df = pd.read_csv(r'..\Data\Preprocessed_data\Platform.csv')
product_df = pd.read_csv(r'..\Data\Preprocessed_data\Product.csv')

# Tiki

#### Product info

In [105]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()
    # Kiểm tra file data có tồn tại hay không
    file_path = r'..\Data\Tiki\%s_info.csv'%('_'.join(product_name.split(' ')))

    if os.path.isfile(file_path):
        tiki_product_info = pd.read_csv(file_path)
    else:
        continue

    # Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
    tiki_product_info['Price'] = tiki_product_info['Price'].str.replace('.', '').astype(int)

    # Tạo và giữ nguyên thông tin bán ở những ngày crawl mới đối với shop không còn bán sp
    for name in tiki_product_info['Name'].unique():
        all_day_crawl = list(tiki_product_info['Crawl date'].unique())
        crawl_date = tiki_product_info[tiki_product_info['Name'] == name]['Crawl date'].values
        check_day = []

        for day in all_day_crawl:
            if day in crawl_date:
                check_day.append(all_day_crawl.index(day))

        if len(check_day) == 1 and check_day[0] != len(tiki_product_info['Crawl date'].unique())-1:
            value = tiki_product_info[(tiki_product_info['Name'] == name) & (tiki_product_info['Crawl date'] == all_day_crawl[check_day[0]])]
            list_df_copy = []
            for index in range(len(tiki_product_info['Crawl date'].unique()[check_day[0]+1:])):
                list_df_copy.append(value.copy())

            for i in range(len(list_df_copy)):
                list_df_copy[i].loc[value.index[0], 'Crawl date'] = all_day_crawl[check_day[0]+1:][i]

            new_date_values = pd.concat(list_df_copy, ignore_index=True)
            tiki_product_info = pd.concat([tiki_product_info, new_date_values], ignore_index=True)
        elif len(check_day) > 1 and len(check_day) != len(all_day_crawl):
            check_list = []

            for i in range(len(check_day) - 1):
                dict_check = {}
                if check_day[i+1] - check_day[i] != 1:
                    dict_check[check_day[i]] = [x for x in range(check_day[i]+1, check_day[i+1])]
                    check_list.append(dict_check)

            if check_day[-1] != len(all_day_crawl)-1:
                check_list.append({check_day[-1]: [x for x in range(check_day[-1]+1, len(all_day_crawl))]})

            new_date_values = pd.DataFrame(columns=tiki_product_info.columns)
            for dict in check_list:
                for k, v in dict.items():
                    value = tiki_product_info[(tiki_product_info['Name'] == name) & (tiki_product_info['Crawl date'] == all_day_crawl[k])]
                    list_df_copy = []
                    for index in range(len(v)):
                        list_df_copy.append(value.copy())
                    
                    for i in range(len(list_df_copy)):
                        list_df_copy[i].loc[value.index[0], 'Crawl date'] = all_day_crawl[v[i]]
                    
                    new_date_values = pd.concat([new_date_values] + list_df_copy, ignore_index=True)

            tiki_product_info = pd.concat([tiki_product_info, new_date_values], ignore_index=True)

    # Thêm cột ID của sp
    tiki_product_info['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    tiki_product_info['PlatformID'] = 'PL01'

    # Thêm shop mới vào file Shop.csv nếu có
    shop_list = tiki_product_info[['Shop name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        tiki_product_info.loc[tiki_product_info['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    # Kiểm tra thiếu cột giá trị
    add_list = ['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Storage', 'Crawl date']
    
    for col in add_list:
        if col not in tiki_product_info.columns:
            tiki_product_info[col] = np.NaN

    add_df = tiki_product_info[add_list].copy()

    # Lưu vào file save Market.csv
    market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    if len(market_df) == 0:
        add_data('Market.csv', add_df, 'MK000')
    else:
        add_data('Market.csv', add_df)

#### Reviews

In [43]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()
    # Kiểm tra file data có tồn tại hay không
    file_path = r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product_name.split(' ')))

    if os.path.isfile(file_path):
        tiki_reviews = pd.read_csv(file_path)
    else:
        continue

    # Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
    actual_date_list = []

    for num in range(len(tiki_reviews)):
        text_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][0]
        crawl_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][1]

        actual_date = extract_date_review(text_date, crawl_date)
        actual_date_list.append(actual_date)

    tiki_reviews['Rating date'] = actual_date_list

    # Biến đổi cột "Rating" thành thang đo likert
    rating_likert = {'Rất không hài lòng': 1, 'Không hài lòng': 2, 'Bình thường': 3, 'Hài lòng': 4, 'Cực kì hài lòng': 5}

    for num in range(len(tiki_reviews)):
        rating = tiki_reviews.loc[num, 'Rating']
        for k, v in rating_likert.items():
            if rating == k:
                tiki_reviews.loc[num, 'Rating'] = v

    # Loại bỏ những dòng trùng "Reviewer name" và "Content"
    tiki_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

    # Thêm cột ID của sp
    tiki_reviews['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    tiki_reviews['PlatformID'] = 'PL01'

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        tiki_reviews.loc[tiki_reviews['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = tiki_reviews[['Content', 'ProductID', 'PlatformID', 'Rating date', 'ShopID', 'Rating']].copy()

    # Lưu vào file save Market.csv
    ecom_reviews_df = pd.read_csv(r'..\Data\Preprocessed_data\EcomReviews.csv')
    if len(ecom_reviews_df) == 0:
        add_data('EcomReviews.csv', add_df, 'ER00000000')
    else:
        add_data('EcomReviews.csv', add_df)

# Lazada

#### Product info

In [107]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()
    # Kiểm tra file data có tồn tại hay không
    file_path = r'..\Data\Lazada\%s_info.csv'%('_'.join(product_name.split(' ')))

    if os.path.isfile(file_path):
        lazada_product_info = pd.read_csv(file_path)
    else:
        continue

    # Chỉnh sửa cột "Quantity" thành số
    if 'int' not in str(lazada_product_info['Quantity'].dtypes):
        for index in range(len(lazada_product_info)):
            count = lazada_product_info.loc[index, 'Quantity']
            sold_count = ''
            x = 0

            if not count.isnumeric():
                if 'k' in count or 'K' in count:
                    x = 1000
                sold_count = count.replace('.', '')
                sold_count = sub('\D', '', count)
                if len(sold_count) == 1:
                    sold_count = int(sold_count) * x
                elif len(sold_count) == 2:
                    sold_count = int(sold_count) * int(x/10)
            else:
                sold_count = int(count)

            lazada_product_info.loc[index, 'Quantity'] = sold_count
            
        for sku in lazada_product_info['SKU'].unique():
            previous_quantity = 0

            for index in lazada_product_info[lazada_product_info['SKU'] == sku].index:
                quantity = lazada_product_info[lazada_product_info['SKU'] == sku].loc[index, 'Quantity']

                if quantity < previous_quantity:
                    lazada_product_info.loc[index, 'Quantity'] = previous_quantity
                else:
                    previous_quantity = quantity
    
    # Tạo và giữ nguyên thông tin bán ở những ngày crawl mới đối với shop không còn bán sp
    for sku in lazada_product_info['SKU'].unique():
        all_day_crawl = list(lazada_product_info['Crawl date'].unique())
        crawl_date = lazada_product_info[lazada_product_info['SKU'] == sku]['Crawl date'].values
        check_day = []

        for day in all_day_crawl:
            if day in crawl_date:
                check_day.append(all_day_crawl.index(day))

        if len(check_day) == 1 and check_day[0] != len(lazada_product_info['Crawl date'].unique())-1:
            value = lazada_product_info[(lazada_product_info['SKU'] == sku) & (lazada_product_info['Crawl date'] == all_day_crawl[check_day[0]])]
            list_df_copy = []
            for index in range(len(lazada_product_info['Crawl date'].unique()[check_day[0]+1:])):
                list_df_copy.append(value.copy())

            for i in range(len(list_df_copy)):
                list_df_copy[i].loc[value.index[0], 'Crawl date'] = all_day_crawl[check_day[0]+1:][i]

            new_date_values = pd.concat(list_df_copy, ignore_index=True)
            lazada_product_info = pd.concat([lazada_product_info, new_date_values], ignore_index=True)
        elif len(check_day) > 1 and len(check_day) != len(all_day_crawl):
            check_list = []

            for i in range(len(check_day) - 1):
                dict_check = {}
                if check_day[i+1] - check_day[i] != 1:
                    dict_check[check_day[i]] = [x for x in range(check_day[i]+1, check_day[i+1])]
                    check_list.append(dict_check)

            if check_day[-1] != len(all_day_crawl)-1:
                check_list.append({check_day[-1]: [x for x in range(check_day[-1]+1, len(all_day_crawl))]})

            new_date_values = pd.DataFrame(columns=lazada_product_info.columns)
            for dict in check_list:
                for k, v in dict.items():
                    value = lazada_product_info[(lazada_product_info['SKU'] == sku) & (lazada_product_info['Crawl date'] == all_day_crawl[k])]
                    list_df_copy = []
                    for index in range(len(v)):
                        list_df_copy.append(value.copy())
                    
                    for i in range(len(list_df_copy)):
                        list_df_copy[i].loc[value.index[0], 'Crawl date'] = all_day_crawl[v[i]]
                    
                    new_date_values = pd.concat([new_date_values] + list_df_copy, ignore_index=True)

            lazada_product_info = pd.concat([lazada_product_info, new_date_values], ignore_index=True)

    # Chỉnh sửa cột "Reviews count" thành số
    for index in range(len(lazada_product_info)):
        review_count = lazada_product_info.loc[index, 'Reviews count']
        if review_count == 'Không có đánh giá':
            lazada_product_info.loc[index, 'Reviews count'] = 0
        else:
            lazada_product_info.loc[index, 'Reviews count'] = int(review_count.replace('đánh giá', ''))

    # Chỉnh sửa cột "Shop rating" về thang đo 5
    for index in range(len(lazada_product_info)):
        shop_rating = lazada_product_info.loc[index, 'Shop rating']

        if shop_rating == 'Không có đánh giá':
            lazada_product_info.loc[index, 'Shop rating'] = ''
        else:
            rating = int(sub('\D', '', shop_rating))
            lazada_product_info.loc[index, 'Shop rating'] = round((5*rating)/100, 1)

    # Thêm cột ID của sp
    lazada_product_info['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    lazada_product_info['PlatformID'] = 'PL02'

    # Thêm shop mới vào file Shop.csv nếu có
    shop_list = lazada_product_info[['Shop name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        lazada_product_info.loc[lazada_product_info['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    # Kiểm tra thiếu cột giá trị
    add_list = ['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Storage', 'Crawl date']
    for col in add_list:
        if col not in lazada_product_info.columns:
            lazada_product_info[col] = np.NaN

    add_df = lazada_product_info[add_list].copy()

    # Lưu vào file save Market.csv
    market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    if len(market_df) == 0:
        add_data('Market.csv', add_df, 'MK000')
    else:
        add_data('Market.csv', add_df)

#### Reviews

In [45]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()

    lazada_reviews = pd.read_csv(r'..\Data\Lazada\%s_reviews.csv'%('_'.join(product_name.split(' '))))

    # Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
    actual_date_list = []

    for num in range(len(lazada_reviews)):
        text_date = lazada_reviews.loc[num, ['Rating date', 'Crawl date']][0]
        if 'trước' in text_date:
            crawl_date = lazada_reviews.loc[num, ['Rating date', 'Crawl date']][1]
            actual_date = extract_date_review(text_date, crawl_date)
        else:
            actual_date = str(datetime.strptime(text_date, '%d thg %m %Y').date())
        actual_date_list.append(actual_date)

    lazada_reviews['Rating date'] = actual_date_list

    # Loại bỏ những dòng trùng "Reviewer name" và "Content"
    lazada_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

    # Thêm cột ID của sp
    lazada_reviews['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    lazada_reviews['PlatformID'] = 'PL01'

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        lazada_reviews.loc[lazada_reviews['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = lazada_reviews[['Content', 'ProductID', 'PlatformID', 'Rating date', 'ShopID', 'Rating']].copy()

    # Lưu vào file save Market.csv
    ecom_reviews_df = pd.read_csv(r'..\Data\Preprocessed_data\EcomReviews.csv')
    if len(ecom_reviews_df) == 0:
        add_data('EcomReviews.csv', add_df, 'ER00000000')
    else:
        add_data('EcomReviews.csv', add_df)

# Shopee

#### Product info

In [109]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()
    # Kiểm tra file data có tồn tại hay không
    file_path = r'..\Data\Shopee\%s_info.csv'%('_'.join(product_name.split(' ')))

    if os.path.isfile(file_path):
        shopee_product_info = pd.read_csv(file_path)
    else:
        continue

    # Tạo lại tên shop
    shopee_product_info['ShopID'] = 'ShopeeID_' + shopee_product_info['ShopID'].astype(str)
    shopee_product_info.rename(columns = {'ShopID':'Shop Name'}, inplace = True)
    shopee_product_info['Shop rating'] = ''
    shop_list = shopee_product_info[['Shop Name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        shopee_product_info.loc[shopee_product_info['Shop Name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    # Price dư 5 số "0"
    shopee_product_info['Price'] = shopee_product_info['Price'].astype(float) / 100000

    # Biến đổi Crawl Date
    shopee_product_info['CrawlDate'] = pd.to_datetime(shopee_product_info['CrawlDate'], format="%Y-%m-%d %H:%M:%S.%f").dt.strftime('%Y-%m-%d')

    # Đổi tên cột
    shopee_product_info.rename(columns = {'CrawlDate':'Crawl date', 'ReviewCount': 'Reviews count', 'StarRating': 'Star rating'}, inplace = True)

    # Kiểm tra thiếu cột giá trị
    add_list = ['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Storage', 'Crawl date']
    for col in add_list:
        if col not in shopee_product_info.columns:
            shopee_product_info[col] = np.NaN

    add_df = shopee_product_info[add_list].copy()

    # Lưu vào file save Market.csv
    market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    if len(market_df) == 0:
        add_data('Market.csv', add_df, 'MK000')
    else:
        add_data('Market.csv', add_df)

#### Reviews

In [ ]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()
    # Kiểm tra file data có tồn tại hay không
    file_path = r'..\Data\Shopee\%s_review.csv'%('_'.join(product_name.split(' ')))

    if os.path.isfile(file_path):
        shopee_reviews = pd.read_csv(file_path)
    else:
        continue

    # Đổi tên shop
    shopee_reviews['ShopID'] = 'ShopeeID_' + shopee_reviews['ShopID'].astype(str)
    shopee_reviews.rename(columns = {'ShopID':'Shop name'}, inplace = True)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        shopee_reviews.loc[shopee_reviews['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    # Đổi tên cột Review thành Content, DateReview thành Rating date, StarRating thành Rating
    shopee_reviews.rename(columns = {'DateReview': 'Rating date', 'StarRating':'Rating'}, inplace = True)

    add_df = shopee_reviews[['Review', 'ProductID', 'PlatformID', 'Rating date', 'ShopID', 'Rating']].copy()

    # Lưu vào file save Market.csv
    ecom_reviews_df = pd.read_csv(r'..\Data\Preprocessed_data\EcomReviews.csv')
    if len(ecom_reviews_df) == 0:
        add_data('EcomReviews.csv', add_df, 'ER00000000')
    else:
        add_data('EcomReviews.csv', add_df)

# Websites

#### Tinhte

# Youtube